In [7]:
!pip install craft-pytorch


ERROR: Could not find a version that satisfies the requirement craft-pytorch (from versions: none)
ERROR: No matching distribution found for craft-pytorch


In [2]:
import sys

sys.path.append("C:/Users/user/Desktop/aws_s3_Data/CRAFT-pytorch-master")
from craft import CRAFT
from craft_utils import getDetBoxes
from imgproc import loadImage

: 

In [1]:
import os
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from craft_pytorch import CRAFT
from craft_pytorch.craft_utils import getDetBoxes
from craft_pytorch.imgproc import loadImage


def load_yolo_model(model_path):
    """Load YOLO model from the given path."""
    model = YOLO(model_path)
    return model


def load_craft_model(weights_path):
    """Load the pre-trained CRAFT model."""
    model = CRAFT()
    model.load_state_dict(torch.load(weights_path, map_location="cpu"))
    model.eval()
    return model


def detect_with_yolo(yolo_model, image, conf_threshold=0.3, iou_threshold=0.4):
    """Detect text and non-text regions using YOLO model."""
    results = yolo_model.predict(
        image, conf=conf_threshold, iou=iou_threshold, verbose=False
    )
    yolo_boxes = results[0].boxes.data  # YOLO bounding boxes
    return yolo_boxes


def detect_with_craft(
    craft_model, image, text_threshold=0.7, link_threshold=0.4, low_text=0.4
):
    """Detect detailed text regions using CRAFT model."""
    image = loadImage(image)  # Normalize image for CRAFT
    with torch.no_grad():
        prediction = craft_model(torch.from_numpy(image).unsqueeze(0))
    score_text, score_link = (
        prediction[0][0, :, :].numpy(),
        prediction[0][1, :, :].numpy(),
    )
    boxes, _ = getDetBoxes(
        score_text, score_link, text_threshold, link_threshold, low_text
    )
    return boxes


def draw_boxes(image, boxes, color, label):
    """Draw bounding boxes on the image."""
    for box in boxes:
        if isinstance(box, (list, np.ndarray)):
            box = np.array(box).astype(np.int32).reshape((-1, 1, 2))
        x1, y1, x2, y2 = map(int, box[:4]) if len(box) == 4 else box.flatten()
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(
            image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2
        )
    return image


def process_image(image_path, yolo_model, craft_model):
    """Process a single image using YOLO and CRAFT models."""
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image at {image_path} could not be loaded.")

    # Step 1: YOLO for text and non-text detection
    yolo_boxes = detect_with_yolo(yolo_model, image)
    text_boxes = []
    non_text_boxes = []

    for box in yolo_boxes:
        x1, y1, x2, y2, conf, cls = box
        if int(cls) == 0:  # Text class
            text_boxes.append([x1, y1, x2, y2])
        elif int(cls) == 1:  # Non-text class
            non_text_boxes.append([x1, y1, x2, y2])

    # Step 2: CRAFT for detailed text detection in YOLO text regions
    detailed_text_boxes = []
    for text_box in text_boxes:
        x1, y1, x2, y2 = map(int, text_box)
        roi = image[y1:y2, x1:x2]
        if roi.size > 0:
            craft_boxes = detect_with_craft(craft_model, roi)
            detailed_text_boxes.extend(craft_boxes)

    # Draw results on the image
    image = draw_boxes(image, text_boxes, color=(0, 255, 0), label="Text (YOLO)")
    image = draw_boxes(
        image, non_text_boxes, color=(255, 0, 0), label="Non-Text (YOLO)"
    )
    image = draw_boxes(
        image, detailed_text_boxes, color=(0, 0, 255), label="Text (CRAFT)"
    )

    return image


def process_folder(input_folder, output_folder, yolo_model, craft_model):
    """Process all images in a folder."""
    os.makedirs(output_folder, exist_ok=True)
    for image_name in os.listdir(input_folder):
        if not image_name.lower().endswith((".jpg", ".jpeg", ".png")):
            continue
        image_path = os.path.join(input_folder, image_name)
        print(f"Processing {image_name}...")
        try:
            processed_image = process_image(image_path, yolo_model, craft_model)
            output_path = os.path.join(
                output_folder, f"{os.path.splitext(image_name)[0]}_result.jpg"
            )
            cv2.imwrite(output_path, processed_image)
            print(f"Saved result: {output_path}")
        except Exception as e:
            print(f"Error processing {image_name}: {e}")


# Paths
yolo_model_path = (
    "C:/Users/user/Desktop/aws_s3_Data/yolov8_text_nontext.pt"  # Path to YOLO model
)
craft_weights_path = (
    "C:/Users/user/Desktop/aws_s3_Data/craft_mlt_25k.pth"  # Path to CRAFT weights
)
input_folder = "./Result/test"  # Folder containing test images
output_folder = "./Result/processed_images"  # Folder to save results

# Load models
yolo_model = load_yolo_model(yolo_model_path)
craft_model = load_craft_model(craft_weights_path)

# Process images in the folder
process_folder(input_folder, output_folder, yolo_model, craft_model)

ModuleNotFoundError: No module named 'craft_pytorch'